In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap

In [2]:
all_samples = pd.read_pickle('/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_read_counts.pickle').dropna(subset=['pos_tube_rack'])

all_samples['RBP'] = all_samples.pos_tube_rack.str[:13]
all_samples['primer_plate'] = all_samples.pos_extraction.str[:13]
all_samples['row'] = all_samples.pos_extraction.str[14]
all_samples['column'] = all_samples.pos_extraction.str[15:].astype('int')

RBP = ['RBP201112A086',
'RBP201112A087',
'RBP201112A088',
'RBP201112A089',
'RBP201112A112',
'RBP201112A113',
'RBP201112A114',
'RBP201112A115']

all_samples = all_samples[all_samples['RBP'].isin(RBP)].sort_values(by='RBP')

## 201222_EXP21_SpittoonStudyAllStars
RBP = ['RBP201112A086',
'RBP201112A087',
'RBP201112A088',
'RBP201112A089',
'RBP201112A112',
'RBP201112A113',
'RBP201112A114',
'RBP201112A115']

In [ ]:
total_cols = 3
total_rows = 8
plt.rcParams['font.size'] = '13'
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, sharex=True, sharey=True, 
                        figsize=(7*total_cols, 5*total_rows), constrained_layout=True, squeeze=False)

fig.suptitle(f'{all_samples.iloc[0,0]}: RNAse Count by RT-PCR Plate, Spittoon Study All Stars')
fig.set_facecolor('white')

for i,k in zip(all_samples.RBP.unique(), range(8)):
    sub_RBP = all_samples[all_samples['RBP'] == i].sort_values(by='primer_plate')
    
    for l,m in zip(sub_RBP.primer_plate.unique(), range(3)):
        sub_primer = sub_RBP[sub_RBP['primer_plate'] == l]
        sub_primer.rnase_count.astype('int64')
        
        plate_map = sub_primer.loc[:, ['row', 'column', 'rnase_count']].pivot(index='row', columns='column', values='rnase_count').fillna(0)
            
        plate_map = plate_map.astype(dtype='int')
        
        row = k % total_rows
        pos = m % total_cols
        
        fig = sns.heatmap(plate_map, annot=True, cbar=False, cmap="Purples", 
                    vmin=500, fmt='d', annot_kws={"fontsize":9}, ax=axs[row][pos])
        
        fig = sns.heatmap(plate_map, mask=plate_map > 50, cmap=ListedColormap(['grey']), annot=False, ax=axs[row][pos], cbar=False)

        axs[row][pos].set_title(f'RBP Plate: {i}, Primer Plate: {l}')
            
plt.savefig('spittoonstudyallstars', dpi=300)


In [ ]:
all_samples[all_samples['run_sample_id'].str.contains(r'\d{14}')]\
.loc[:, ['run_sample_id', 'RBP', 'row', 'column']].sort_values(by=['RBP', 'row', 'column']).drop_duplicates()

## Compare Saliva and Nasal Pos Control G19 Score

In [ ]:
calls = pd.read_pickle('/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_read_counts.pickle').dropna(subset=['pos_tube_rack'])
samples = calls[calls['run_sample_id'].str.startswith(('G', 'H'), na=False)]
samples = samples[~samples['run_sample_id'].str.startswith('Ht', na=False)]

clin_rbp = samples[samples['runid'].str.contains(r'^2012.*$')].pos_tube_rack.str.split(':').str[0].unique()

clin_pos = calls[calls['pos_tube_rack'].str.contains('|'.join(clin_rbp))].query("sample_type == 'Positive'").replace({'Positive':'nasal_pos'})
saliva_pos = all_samples.query("sample_type == 'Positive'").replace({'Positive':'saliva_pos'})

both = pd.concat([clin_pos, saliva_pos])

In [ ]:
matplotlib.rcParams.update({'font.size': 15})

a = sns.boxplot(x='sample_type', y='covid_ratio', data=both, showfliers=False, palette="Purples")
a.set_yscale('log')

plt.axhline(y=0.01, color='r', linewidth=1, ls='--')
plt.ylim(0,1)
plt.ylabel('G19 Score')

plt.title('Positive Control G19 Score Comparison\nBetween Nasal and Saliva', y=1.05)
plt.show()